## 加载Keras中的MNIST数据集

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.
/home/zjk/data/anaconda3/envs/tf_1.11.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zjk/data/anaconda3/envs/tf_1.11.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zjk/data/anaconda3/envs/tf_1.11.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home

图像被编码为Numpy数组,而标签是数字数组,取值范围为0~9.图像和标签一一对应.

## 来看一下训练数据

In [2]:
train_images.shape

(60000, 28, 28)

通过`train_images.shape`属性可以看到,张量是**三维**的,代表了有60000个样本,28x28像素.

In [3]:
len(train_labels)

60000

## 下面是测试数据：

In [4]:
test_images.shape

(10000, 28, 28)

In [5]:
len(test_labels)

10000

In [6]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

接下来的工作流程如下:首先,将训练数据输入神经网络;其次,网络学习将图像与标签关联在一起;最后,网络对test_images生成预测,而我们将验证这些预测与test_labels中的标签是否匹配.

## 下面来构建网络

In [7]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

神经网络的核心组件是**层（layer）**,他是一种数据处理模块,你可以将它看成数据过滤器.
本例的网络包含2个Dense层,他们是密集连接（也叫**全连接**）的神经层.第二层是一个10路softmax层,他讲返回一个由10个概率值（总和为1）组成的数组。每个概率值表示当前数字图像属于10个数字类别中某一个的概率。

## 编译步骤

要想训练网络，我们还需要选择编译（compile）步骤的三个参数.
 * **损失函数（loss function）**：网络如何衡量在训练数据上的性能，即网络如何朝着正确的方向前进。
 * **优化器（optimizer）**：基于训练数据和损失函数来更新网络的机制。
 * **在训练和测试过程中需要监控的指标（metric）**：本例只关心精度，即正确分类的图像所占的比例。

In [8]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

## 对数据进行预处理

在开始训练之前，我们将对数据进行预处理，将其变换为网络要求的形状，并缩放到所
有值都在 [0, 1] 区间。比如，之前训练图像保存在一个 uint8 类型的数组中，其形状为
(60000, 28, 28)，取值区间为 [0, 255]。我们需要将其变换为一个 float32 数组，其形
状为 (60000, 28 * 28)，取值范围为 0~1

In [10]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

还需要对标签进行分类编码

In [11]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## **拟合（fit）**模型

In [13]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.2585 - acc: 0.9257
Epoch 2/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.1040 - acc: 0.9698
Epoch 3/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.0679 - acc: 0.9796
Epoch 4/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.0489 - acc: 0.9855
Epoch 5/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.0363 - acc: 0.9891


训练过程中显示了两个数字：一个是网络在训练数据上的损失（loss）,另一个是网络在训练数据上的精度（acc）.

## 检查模型在测试集上的性能

In [15]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

10000/10000 [==============================] - 0s 26us/step
test_acc: 0.9794
